In [0]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##  Reading a sample of Data from both train and test

In [0]:
import random
if os.path.isfile('/content/drive/My Drive/FacebookFriend/data/after_eda/train_after_eda.csv'):
    filename = "/content/drive/My Drive/FacebookFriend/data/after_eda/train_after_eda.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 15100030
    # n_train = sum(1 for line in open(filename)) #number of records in file (excludes header)
    n_train =  15100028
    s = 100000 #desired sample size
    skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039

In [0]:
if os.path.isfile('/content/drive/My Drive/FacebookFriend/data/after_eda/train_after_eda.csv'):
    filename = "/content/drive/My Drive/FacebookFriend/data/after_eda/test_after_eda.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 3775008
    # n_test = sum(1 for line in open(filename)) #number of records in file (excludes header)
    n_test = 3775006
    s = 50000 #desired sample size
    skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))
    #https://stackoverflow.com/a/22259008/4084039

In [5]:
print("Number of rows in the train data file:", n_train)
print("Number of rows we are going to elimiate in train data are",len(skip_train))
print("Number of rows in the test data file:", n_test)
print("Number of rows we are going to elimiate in test data are",len(skip_test))

Number of rows in the train data file: 15100028
Number of rows we are going to elimiate in train data are 15000028
Number of rows in the test data file: 3775006
Number of rows we are going to elimiate in test data are 3725006


In [6]:
df_final_train = pd.read_csv('/content/drive/My Drive/FacebookFriend/data/after_eda/train_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv('/content/drive/My Drive/FacebookFriend/data/train_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100002, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,1814022,1791177,1


In [7]:
df_final_test = pd.read_csv('/content/drive/My Drive/FacebookFriend/data/after_eda/test_after_eda.csv', skiprows=skip_test, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('/content/drive/My Drive/FacebookFriend/data/test_y.csv', skiprows=skip_test, names=['indicator_link'])
print("Our test matrix size ",df_final_test.shape)
df_final_test.head(2)

Our test matrix size  (50002, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,210836,1616872,1


In [8]:
if os.path.isfile('/content/drive/My Drive/FacebookFriend/data/after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist('/content/drive/My Drive/FacebookFriend/data/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))
else:
    print("please run the FB_EDA.ipynb or download the files from drive")

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


In [0]:
#for followees
def preferential_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a))*(set(train_graph.successors(b)))))
    except:
        return 0
    return sim

In [25]:
#one test case
print(preferential_for_followees(273084,1505602))

0


In [26]:
#node 1635354 not in graph 
print(preferential_for_followees(273084,1635354))

0


In [0]:
#for followers
def preferential_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a))*(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [28]:
print(preferential_for_followers(273084,470294))

0


In [29]:
#node 1635354 not in graph 
print(preferential_for_followers(669354,1635354))

0


In [0]:
if not os.path.isfile('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage5.h5'):
    #mapping jaccrd followers to train and test data
    df_final_train['preferential_followers'] = df_final_train.apply(lambda row:
                                            preferential_for_followers(row['source_node'],row['destination_node']),axis=1)
    df_final_test['preferential_followers'] = df_final_test.apply(lambda row:
                                            preferential_for_followers(row['source_node'],row['destination_node']),axis=1)

    #mapping jaccrd followees to train and test data
    df_final_train['preferential_followees'] = df_final_train.apply(lambda row:
                                            preferential_for_followees(row['source_node'],row['destination_node']),axis=1)
    df_final_test['preferential_followees'] = df_final_test.apply(lambda row:
                                            preferential_for_followees(row['source_node'],row['destination_node']),axis=1)
    
    hdf = HDFStore('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage5.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage5.h5', 'train_df',mode='r')
    df_final_test = read_hdf('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage5.h5', 'test_df',mode='r')

In [0]:
# df_final_train.head()

# SVD_dot product between source node and destination node feature

In [0]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [0]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=6, n_iter=7, random_state=42)
svd_mat = svd.fit_transform(Adj)

In [0]:
nodelist=sorted(train_graph.nodes())
nodes_list_dict = {k:v for v,k in enumerate(nodelist)}

In [12]:
from tqdm import tqdm
svd_dot = []
for id,temp in tqdm(df_final_train.iterrows(),total=df_final_train.shape[0]):
    in_idx = nodes_list_dict.get(temp.destination_node,'X')
    out_idx = nodes_list_dict.get(temp.source_node,'X')
    
    if ( in_idx != 'X' and out_idx != 'X' ):
        #dot product of svd vector of Source and destination
        svd_temp = np.dot(svd_mat[in_idx,:],svd_mat[out_idx,:])
        svd_dot.append(svd_temp)
    else:
        svd_dot.append(0)

100%|██████████| 100002/100002 [00:13<00:00, 7143.29it/s]


In [0]:
import pickle
pickle.dump(svd_dot,open('/content/drive/My Drive//FacebookFriend/data/svd_dot_train.p','wb'))

In [0]:
df_final_train['svd_dot'] = svd_dot

In [34]:
df_final_train.head(5)

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,...,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,preferential_followers,preferential_followees,svd_dot
0,273084,1505602,1,0,0.000000,0.000000,0.000000,6,15,8,...,3.459963e-07,0.000773,0.000756,1.943132e-13,1.941103e-13,9.226339e-16,2.231877e-15,0,0,1.183038e-07
1,832016,1543415,1,0,0.187135,0.028382,0.343828,94,61,142,...,6.427660e-07,0.000845,0.001317,3.906648e-11,9.424102e-11,1.208074e-11,1.273080e-10,0,0,4.244122e-39
2,1325247,760242,1,0,0.369565,0.156957,0.566038,28,41,22,...,5.179801e-07,0.000885,0.000855,7.730764e-114,4.067322e-114,2.681298e-113,2.199205e-113,0,0,1.860887e-15
3,1368400,1006992,1,0,0.000000,0.000000,0.000000,11,5,7,...,1.704245e-06,0.000739,0.000773,5.443738e-17,4.139999e-16,2.413250e-14,6.688064e-15,0,0,-9.541665e-32
4,140165,1708748,1,0,0.000000,0.000000,0.000000,1,11,3,...,2.089590e-07,0.000751,0.000735,3.887821e-16,4.721269e-16,7.552255e-16,2.734009e-18,0,0,7.750748e-22


In [16]:
svd_dot_test = []
for idx,temp in tqdm(df_final_test.iterrows(),total=df_final_test.shape[0]):
    in_idx = nodes_list_dict.get(temp.destination_node,'X')
    out_idx = nodes_list_dict.get(temp.source_node,'X')
    
    if ( in_idx != 'X' and out_idx != 'X' ):
        svd_temp = np.dot(svd_mat[in_idx,:],svd_mat[out_idx,:])
        svd_dot_test.append(svd_temp)
    else:
        svd_dot_test.append(0)

100%|██████████| 50002/50002 [00:06<00:00, 7628.82it/s]


In [0]:
pickle.dump(svd_dot_test,open('/content/drive/My Drive//FacebookFriend/data/svd_dot_test.p','wb'))

In [0]:
df_final_test['svd_dot'] = svd_dot_test

In [36]:
df_final_test.head(5)

,source_node,destination_node,indicator_link,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,...,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,preferential_followers,preferential_followees,svd_dot
0,848424,784690,1,0,0.0,0.029161,0.000000,14,6,9,...,0.000002,0.000754,0.000786,3.243237e-16,1.745627e-16,2.969838e-15,9.269213e-14,0,0,8.908217e-16
1,483294,1255532,1,0,0.0,0.000000,0.000000,17,1,19,...,0.000001,0.000739,0.000801,1.702625e-19,2.706300e-15,2.522357e-16,5.277458e-15,0,0,0.000000e+00
2,626190,1729265,1,0,0.0,0.000000,0.000000,10,16,9,...,0.000002,0.000789,0.000770,9.426796e-14,4.116616e-16,2.253244e-15,2.079387e-15,0,0,9.989362e-27
3,947219,425228,1,0,0.0,0.000000,0.000000,37,10,34,...,0.000002,0.000778,0.000884,9.876114e-14,1.039593e-13,1.511694e-14,3.478438e-14,0,0,1.899801e-06
4,991374,975044,1,0,0.2,0.042767,0.347833,27,15,27,...,0.000001,0.000779,0.000840,1.557332e-15,1.096037e-14,5.180869e-15,1.296135e-14,0,0,2.095827e-19


In [0]:
from pandas import HDFStore,DataFrame

In [0]:
hdf = HDFStore('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage6.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.put('test_df',df_final_test, format='table', data_columns=True)
hdf.close()

In [0]:
from pandas import read_hdf
df_final_train = read_hdf('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage6.h5', 'train_df',mode='r')
df_final_test = read_hdf('/content/drive/My Drive/FacebookFriend/data/fea_sample/storage_sample_stage6.h5', 'test_df',mode='r')

In [0]:
y_train = df_final_train.indicator_link 
y_test = df_final_test.indicator_link

In [0]:
df_final_train.drop(['source_node', 'destination_node','indicator_link'],axis=1,inplace=True)
df_final_test.drop(['source_node', 'destination_node','indicator_link'],axis=1,inplace=True)

In [41]:
df_final_train.columns

Index(['jaccard_followers', 'jaccard_followees', 'cosine_followers',
       'cosine_followees', 'num_followers_s', 'num_followees_s',
       'num_followees_d', 'inter_followers', 'inter_followees', 'adar_index',
       'follows_back', 'same_comp', 'shortest_path', 'weight_in', 'weight_out',
       'weight_f1', 'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s',
       'page_rank_d', 'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'preferential_followers', 'preferential_followees',
       'svd_dot'],
      dtype='object')

In [42]:
df_final_train.head(2)

,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followees_s,num_followees_d,inter_followers,inter_followees,adar_index,...,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,preferential_followers,preferential_followees,svd_dot
0,0,0.000000,0.000000,0.000000,6,15,8,0,0,0.000000,...,3.459963e-07,0.000773,0.000756,1.943132e-13,1.941103e-13,9.226339e-16,2.231877e-15,0,0,1.183038e-07
1,0,0.187135,0.028382,0.343828,94,61,142,11,32,16.362912,...,6.427660e-07,0.000845,0.001317,3.906648e-11,9.424102e-11,1.208074e-11,1.273080e-10,0,0,4.244122e-39


In [43]:
df_final_test.columns

Index(['jaccard_followers', 'jaccard_followees', 'cosine_followers',
       'cosine_followees', 'num_followers_s', 'num_followees_s',
       'num_followees_d', 'inter_followers', 'inter_followees', 'adar_index',
       'follows_back', 'same_comp', 'shortest_path', 'weight_in', 'weight_out',
       'weight_f1', 'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s',
       'page_rank_d', 'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'preferential_followers', 'preferential_followees',
       'svd_dot'],
      dtype='object')